## In-Sample Analysis 

**Phase 1: Model Estimation (Full Sample)**  
(Goal: Estimate parameters for your primary model and all benchmark models using the entire dataset)

### Load Data:
- Load the preprocessed `df_vw_returns_final` (N=95, T=726) into your Python environment.

In [7]:
import polars as pl
import numpy as np
import os
from bellman_filter_dfsv.filters.bellman_information import DFSVBellmanInformationFilter
from bellman_filter_dfsv.models.dfsv import DFSVParamsDataclass
from bellman_filter_dfsv.utils.optimization import run_optimization
from bellman_filter_dfsv.filters.particle import DFSVParticleFilter
from bellman_filter_dfsv.utils.optimization import FilterType
from bellman_filter_dfsv.utils.optimization_helpers import create_stable_initial_params
import time
import jax
import jax.numpy as jnp
import cloudpickle
import pickle
from pathlib import Path
from model_fitting import save_result
df =pl.read_csv("../vw_returns_final.csv")
df
N=95
K=5
jax.config.update("jax_enable_x64", True)
returns_jax=df.to_jax()
returns_jax.dtype

dtype('float64')

In [2]:
#Initial parameter guess
def create_realistic_initial_params(
    N: int,
    K: int,
    portfolio_sample_variances: np.ndarray # Accept numpy array for convenience
    ) -> DFSVParamsDataclass:
    """
    Creates realistic initial parameter guesses for the DFSV model optimization using JAX.

    Args:
        N: Number of observed series (assets).
        K: Number of latent factors.
        portfolio_sample_variances: A NumPy array containing the full-sample variance
                                     for each of the N portfolio return series.

    Returns:
        DFSVParamsDataclass: Initial parameter values suitable for starting optimization.
    """
    if len(portfolio_sample_variances) != N:
        raise ValueError(f"Length of portfolio_sample_variances ({len(portfolio_sample_variances)}) must match N ({N})")

    # Convert input variances to JAX array
    portfolio_sample_variances_jax = jnp.array(portfolio_sample_variances, dtype=jnp.float64)

    # --- 1. Factor Loadings (lambda_r) ---
    # Initialize as zeros
    lambda_r_init = jnp.zeros((N, K), dtype=jnp.float64)
    # Set the top KxK block to lower triangular with diagonal 1s for identification
    # Use min(N, K) in range for safety if N < K (though unlikely here)
    for i in range(min(N, K)):
        lambda_r_init = lambda_r_init.at[i, i].set(1.0) # Diagonal = 1
        # Ensure upper triangle is zero within the KxK block
        for j in range(i + 1, K):
            lambda_r_init = lambda_r_init.at[i, j].set(0.0) # Upper triangle = 0

    # --- 2. Factor Transition Matrix (Phi_f) ---
    # Start with moderate diagonal persistence and small off-diagonal noise
    diag_phi_f = 0.5
    off_diag_phi_f = 0.01 # Small value for off-diagonal elements
    phi_f_init = jnp.full((K, K), off_diag_phi_f, dtype=jnp.float64)
    phi_f_init = phi_f_init.at[jnp.diag_indices(K)].set(diag_phi_f)
    # Simple stability check/adjustment using JAX eigenvalues
    # Ensure eigenvalues are within unit circle - this simple scaling might not be perfect for larger K
    eigenvalues_f = jnp.linalg.eigvals(phi_f_init) # Use JAX eigvals
    max_eig_f = jnp.max(jnp.abs(eigenvalues_f))
    # Use jax.lax.cond for conditional execution if needed, or simple check
    # Note: Direct comparison might cause issues in JIT context, but okay for initialization
    if max_eig_f >= 0.99: # Use 0.99 to be safe
        print(f"Warning: Initial Phi_f max eigenvalue {max_eig_f:.3f} >= 0.99. Scaling down.")
        phi_f_init = phi_f_init * (0.98 / max_eig_f)

    # --- 3. Log-Volatility Transition Matrix (Phi_h) ---
    # Start with high diagonal persistence, assume independence initially
    diag_phi_h = 0.98
    phi_h_init = jnp.eye(K, dtype=jnp.float64) * diag_phi_h
    # Stability check (already diagonal and < 1, so likely stable)
    eigenvalues_h = jnp.linalg.eigvals(phi_h_init) # Use JAX eigvals
    max_eig_h = jnp.max(jnp.abs(eigenvalues_h))
    if max_eig_h >= 0.99: # Should not trigger for diagonal 0.98
        print(f"Warning: Initial Phi_h max eigenvalue {max_eig_h:.3f} >= 0.99. Scaling down.")
        phi_h_init = phi_h_init * (0.98 / max_eig_h)

    # --- 4. Log-Volatility Mean Vector (mu) ---
    # Based on typical monthly volatility -> log(sigma^2)
    mu_init = jnp.full(K, -6.0, dtype=jnp.float64)

    # --- 5. Log-Volatility Innovation Covariance (Q_h) ---
    # Diagonal matrix, assuming std dev of log-vol shocks ~ 0.2 -> variance ~ 0.04
    q_h_diag_val = 0.04
    q_h_init = jnp.eye(K, dtype=jnp.float64) * q_h_diag_val

    # --- 6. Idiosyncratic Variances (sigma2 - diagonal of Sigma_epsilon) ---
    # Based on sample variance and proportion unexplained by PCA (e.g., 15%)
    proportion_idiosyncratic = 0.15
    # Ensure variances are positive, use maximum with a small floor
    # Use the JAX array version of sample variances
    sigma2_init = jnp.maximum(proportion_idiosyncratic * portfolio_sample_variances_jax, 1e-8)

    # Create the dataclass instance
    initial_params = DFSVParamsDataclass(
        N=N,
        K=K,
        lambda_r=lambda_r_init,
        Phi_f=phi_f_init,
        Phi_h=phi_h_init,
        mu=mu_init,
        sigma2=sigma2_init, # This is the vector of diagonal variances
        Q_h=q_h_init
    )

    return initial_params
sample_var=jnp.var(returns_jax,axis=0)
# print(sample_var)
init_params=create_realistic_initial_params(N,K,sample_var)
init_params

DFSVParamsDataclass(N=95, K=5, lambda_r=Array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
   

### Estimate DFSV-BIF Model:
- Define the DFSV model structure (N=95, K=5, VAR(1), lower-triangular Λ constraint).
- Set initial parameter values.
- Run BIF-based MLE (e.g., DampedTrustRegionBFGS) on t=1 to 726.

In [6]:
#Initial parameters


#NOTE: takes around 3hrs to run, load pickle file 
# bif_filter=DFSVBellmanInformationFilter(N,K)
# # Run optimization with BIF
# start_time = time.time()
# results_bif = run_optimization(
#     filter_type=FilterType.BIF,
#     returns=returns_jax,  # Your returns data
#     initial_params=init_params,
#     optimizer_name="DampedTrustRegionBFGS",  
#     use_transformations=True,  # Enable parameter transformations for better stability
#     max_steps=500,  # Reasonable number of steps based on your examples
#     stability_penalty_weight=1000.0,  # Default penalty weight from your codebase
#     verbose=True,
#     log_params=False,  # Enable parameter logging for analysis
#     log_interval=1,
#     fix_mu=False
# )

# Load results
bif_pkl = "bif_full_result_20250425_144625.pkl"
with open(bif_pkl, "rb") as f:
    results_bif = cloudpickle.load(f)
print(results_bif)
# print(theta_bif, loglik_bif, time_bif, conv_bif)

time_bif = results_bif.time_taken
theta_bif = results_bif.final_params  # Final optimized parameters
loglik_bif = results_bif.final_loss  # Extract final pseudo-likelihood
conv_bif = results_bif.success  # Convergence status
print(f"Estimated parameters: {theta_bif}")
print(f"BIF Optimization completed in {time_bif:.2f} seconds")
print(f"Convergence status: {conv_bif}")
print(f"Final log-likelihood: {loglik_bif:.4f}")

OptimizerResult(filter_type=<FilterType.BIF: 1>, optimizer_name='DampedTrustRegionBFGS', uses_transformations=True, fix_mu=False, prior_config_name='No Priors', success=Array(True, dtype=bool), result_code=optimistix._solution.RESULTS<>, final_loss=Array(-222084.10371758, dtype=float64), steps=Array(289, dtype=int64, weak_type=True), time_taken=10511.067908525467, error_message=None, final_params=DFSVParamsDataclass(N=95, K=5, lambda_r=Array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 8.64240933e-01,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 9.44707338e-01, -8.46640119e-02,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 7.96250567e-01,  1.99145606e-02, -9.74564516e-03,
         1.00000000e+00,  0.00000000e+00],
       [ 8.18062280e-01,  1.74249316e-02,  1.00618529e-01,
         2.76201051e-01,  1.00000000e+00],
       [ 7.39165123e-01, -4.19852043e-02,  2.81170

### Record Metrics (DFSV-BIF):
- Estimated parameters `Θ^_BIF`
- Final pseudo-log-likelihood `L_BIF`
- Number of free parameters `p_DFSV`
- Total estimation time
- Convergence status/flags

In [ ]:
#load pickle file


bif_params=results_bif.final_params

OptimizerResult(filter_type=<FilterType.BIF: 1>, optimizer_name='DampedTrustRegionBFGS', uses_transformations=True, fix_mu=False, prior_config_name='No Priors', success=Array(False, dtype=bool), result_code=optimistix._solution.RESULTS<The maximum number of steps was reached in the nonlinear solver. The problem may not be solveable (e.g., a root-find on a function that has no roots), or you may need to increase `max_steps`.>, final_loss=Array(-76371.57597898, dtype=float64), steps=Array(10, dtype=int64, weak_type=True), time_taken=64.89801907539368, error_message=None, final_params=DFSVParamsDataclass(N=95, K=5, lambda_r=Array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0.,

In [ ]:
#Save result

# # Create output directory
# output_dir = Path("outputs/empirical/insample")
# output_dir.mkdir(parents=True, exist_ok=True)
# save_result(result_bif, output_dir)

Full result object saved to outputs/empirical/insample/bif_full_result_20250425_144625.pkl


### Estimate DFSV-PF Model (Benchmark 1):
- Define DFSV model structure (same as BIF).
- Set initial params, specify particles (e.g., P=10000).
- Run PF-based MLE (e.g., ArmijoBFGS).

In [ ]:
# # Run optimization with PF
start_time = time.time()
results_pf = run_optimization(
    filter_type=FilterType.PF,
    returns=returns_jax,  # Your returns data
    initial_params=init_params,
    optimizer_name="ArmijoBFGS",  
    use_transformations=True,  # Enable parameter transformations for better stability
    max_steps=1000,  # Reasonable number of steps based on your examples
    stability_penalty_weight=1000.0,  # Default penalty weight from your codebase
    verbose=True,
    log_params=False,  # Enable parameter logging for analysis
    log_interval=1,
    fix_mu=False
)
#save results
output_dir = Path("outputs")
output_dir.mkdir(parents=True, exist_ok=True)
save_result(results_pf, output_dir,filter_type="PF")

DFSVParamsDataclass(N=95, K=5, lambda_r=Array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
   

### Record Metrics (DFSV-PF):
- Estimated parameters `Θ^_PF`
- Final log-likelihood `L_PF`
- Number of free parameters `p_DFSV`
- Total estimation time
- Convergence status/flags

In [10]:
# TODO: Display DFSV-PF results
print(theta_pf, loglik_pf, time_pf, conv_pf)

Ellipsis Ellipsis Ellipsis Ellipsis


In [9]:
df

SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,ME1 BM5,ME1 BM6,ME1 BM7,ME1 BM8,ME1 BM9,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,ME2 BM5,ME2 BM6,ME2 BM7,ME2 BM8,ME2 BM9,ME2 BM10,ME3 BM1,ME3 BM2,ME3 BM3,ME3 BM4,ME3 BM5,ME3 BM6,ME3 BM7,ME3 BM8,ME3 BM9,ME3 BM10,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,ME4 BM6,ME4 BM7,…,ME6 BM9,ME6 BM10,ME7 BM1,ME7 BM2,ME7 BM3,ME7 BM4,ME7 BM5,ME7 BM6,ME7 BM7,ME7 BM8,ME7 BM9,ME8 BM1,ME8 BM2,ME8 BM3,ME8 BM4,ME8 BM5,ME8 BM6,ME8 BM7,ME8 BM8,ME8 BM9,ME8 BM10,ME9 BM1,ME9 BM2,ME9 BM3,ME9 BM4,ME9 BM5,ME9 BM6,ME9 BM7,ME9 BM8,ME9 BM9,BIG LoBM,ME10 BM2,ME10 BM3,ME10 BM4,ME10 BM5,ME10 BM6,ME10 BM7
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.007382,-0.00115,0.000195,-0.012834,0.011184,-0.009643,-0.020203,-0.02312,-0.028236,-0.024615,0.001038,0.009439,-0.028251,-0.005772,-0.014653,0.001929,-0.007548,0.00123,-0.023592,-0.035555,-0.036638,-0.012083,-0.035418,-0.038257,-0.028587,0.000576,-0.032932,-0.038599,-0.018503,-0.043903,-0.021401,-0.03328,0.04106,-0.007844,-0.021999,-0.028473,-0.036914,…,-0.037197,-0.022314,-0.019794,-0.015607,-0.005665,-0.028831,-0.015313,-0.018285,-0.027472,-0.055769,-0.00893,-0.019607,-0.0224,-0.025628,-0.038781,-0.031649,-0.048871,-0.030455,-0.009043,-0.008621,-0.044927,-0.00355,-0.012611,-0.029586,-0.008267,-0.014858,0.002703,-0.019921,-0.00038,-0.009827,-0.01727,0.0054,-0.007834,0.008023,0.010752,-0.008536,0.012789
-0.015445,0.056861,0.029593,-0.032771,-0.001632,0.000515,0.002825,0.02111,0.032894,0.037321,0.035657,0.066554,0.019937,-0.006961,0.014189,-0.002376,0.008009,0.008977,0.009878,0.047228,0.01445,0.03831,-0.002914,0.031265,0.035098,0.004115,0.035053,0.030702,0.040508,0.086509,0.088688,0.034431,0.022356,0.068553,0.059914,0.021251,0.027649,…,0.028936,0.040969,0.047108,0.05369,0.019825,0.030576,0.049609,0.07058,0.0472,0.08927,-0.001574,0.054106,0.025933,0.052941,0.036128,0.051003,0.036179,0.031741,0.08445,0.020393,0.080759,0.055587,0.059238,0.042565,0.032776,0.043458,0.020987,0.067288,0.060346,0.06246,0.049175,0.042147,0.035406,0.02715,0.031177,0.063951,0.05092
0.032145,-0.071611,0.004897,0.014082,0.002553,-0.031038,-0.003224,-0.037328,-0.04369,-0.016865,-0.058385,-0.040669,-0.0112,-0.029059,-0.016086,-0.035482,-0.02907,-0.040419,-0.003016,-0.017765,-0.039637,-0.065549,-0.04261,-0.052996,-0.026429,-0.028344,0.007662,-0.033396,-0.049269,-0.032242,-0.048274,-0.049032,-0.018408,-0.001363,-0.017452,-0.015387,-0.031552,…,-0.057473,-0.03975,-0.054023,-0.034605,-0.023108,-0.054679,-0.042428,-0.029575,-0.030159,-0.062835,-0.010228,-0.050546,-0.018176,-0.041364,-0.013859,-0.026435,-0.02054,-0.026251,-0.078787,-0.052063,-0.032255,-0.030594,-0.029629,-0.042851,-0.028297,-0.031589,0.007768,-0.041571,-0.053297,-0.034704,-0.014703,-0.033533,-0.00263,-0.034347,-0.017656,-0.032772,-0.016334
-0.018672,0.075945,0.010356,-0.0164,0.020624,-0.032454,-0.006044,0.008798,-0.008982,-0.006307,-0.019903,0.013629,-0.01447,-0.058665,0.009258,0.010388,-0.033247,-0.018367,0.043393,0.010961,-0.021657,0.005591,0.010761,0.041551,0.013149,-0.003491,0.030012,0.02131,-0.001799,0.019706,0.033043,-0.010117,0.041718,0.025571,0.030163,0.000488,-0.019864,…,-0.00336,-0.039736,0.004933,-0.019803,-0.025259,0.003941,0.045151,0.017299,-0.00582,0.058881,0.002859,-0.000691,-0.034187,0.001479,-0.000448,-0.00627,0.028212,0.009682,0.080374,0.043658,-0.040812,0.023175,0.015611,-0.012549,-0.00487,0.001723,-0.015878,0.057365,0.002747,-0.00497,0.029801,0.073885,0.010673,0.010661,-0.020136,0.007473,-0.018532
0.007482,-0.084702,-0.037264,-0.051569,-0.030784,-0.027382,-0.032014,-0.025378,-0.028168,-0.032286,-0.065935,-0.022185,-0.053011,-0.049747,-0.029252,-0.030737,-0.013174,-0.024406,-0.007009,-0.031363,-0.056806,-0.068095,-0.042912,-0.010428,-0.032772,0.011225,0.016007,-0.017036,-0.019842,-0.0075

### Estimate DCC-GARCH Model (Benchmark 2):
- Use `arch` library for univariate GARCH(1,1).
- Specify DCC(1,1) model.
- Fit combined DCC-GARCH on t=1 to 726.

In [ ]:
# Fit DCC-GARCH model using arch package
import arch
from arch.univariate import GARCH
from arch.univariate import Normal
import numpy as np
from arch.univariate import ARCHModelResult
from time import time

start_time = time()
returns=df
# 1. Fit univariate GARCH(1,1) models to each series
n_series = returns.shape[1]
garch_fits = []
std_resids = np.zeros_like(returns)

for i in range(n_series):
    model = GARCH(p=1, q=1, dist=Normal())
    result = model.fit(returns[:, i], disp='off')
    garch_fits.append(result)
    std_resids[:, i] = result.resid / result.conditional_volatility

# 2. Estimate DCC parameters using standardized residuals

dcc_model = arch.arch_model(std_resids, vol='DVECH', p=1, q=1)
dcc_result = dcc_model.fit(update_freq=0, disp='off')

# Store results
theta_dcc = dcc_result.params  # DCC parameters
loglik_dcc = dcc_result.loglikelihood  # Log-likelihood
time_dcc = time() - start_time  # Computation time
conv_dcc = dcc_result.convergence_flag  # Convergence status

ImportError: cannot import name 'ARCHModelResult' from 'arch.univariate' (/home/givanib/Documents/BellmanFilterDFSV/.venv/lib/python3.13/site-packages/arch/univariate/__init__.py)

### Record Metrics (DCC-GARCH):
- Estimated parameters `Θ^_DCC`
- Final log-likelihood `L_DCC`
- Number of free parameters `p_DCC`
- Total estimation time
- Convergence status/flags

In [12]:
# TODO: Display DCC-GARCH results
print(theta_dcc, loglik_dcc, time_dcc, conv_dcc)

Ellipsis Ellipsis Ellipsis Ellipsis


### Estimate Factor-CV Model (Benchmark 3):
- (Optional) PCA pre-step for initial Λ and f_t with K=5.
- Define state-space model in `statsmodels.tsa.statespace`.
- Fit model using Kalman Filter MLE.

In [13]:
# TODO: Define and fit Factor-CV state-space model
theta_cv = ...
loglik_cv = ...
time_cv = ...
conv_cv = ...

### Record Metrics (Factor-CV):
- Estimated parameters `Θ^_CV`
- Final log-likelihood `L_CV`
- Number of free parameters `p_CV`
- Total estimation time
- Convergence status/flags